In [86]:
import rasterio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sys
from pathlib import Path

print("Starting final composite plot generation...")

Starting final composite plot generation...


In [87]:
# --- Directories ---
BASE_DIR = Path.cwd().parent.parent
DATA_DIR = BASE_DIR / "data"
RESULTS_DIR = BASE_DIR / "results"

# --- 1. Define File Paths and Thresholds ---
RESISTANCE_RASTER = RESULTS_DIR / "final_resistance_surface.tif"
TRAFFIC_RASTER = RESULTS_DIR / "final_corridor_traffic.tif"
# This is the map of costs *only* on the corridors
BOTTLENECK_RASTER = RESULTS_DIR / "corridor_bottlenecks.tif"
OUTPUT_PLOT = RESULTS_DIR / "final_annotated_composite_map.png"

# --- Analysis Settings ---
# Set the cost value above which we draw a circle
# Since your barrier is 10000, 5000 is a good start
BOTTLENECK_COST_THRESHOLD = 5000.0 
# Set the percentile of traffic to show (e.g., top 1%)
TRAFFIC_PERCENTILE_THRESHOLD = 99.0
# Your barrier cost and the NoData value from previous scripts
EXTREME_BARRIER_COST = 10000.0
NODATA_VALUE = -9999.0

In [88]:
# --- 2. Load Base Resistance Raster (for plotting) ---
print(f"Loading base resistance raster: {RESISTANCE_RASTER}")
with rasterio.open(RESISTANCE_RASTER) as src:
    meta = src.meta.copy()
    resistance_array = src.read(1).astype(np.float32)
    # Apply the same robust cleaning as the worker script
    resistance_array = np.nan_to_num(
        resistance_array, 
        nan=EXTREME_BARRIER_COST,
        posinf=EXTREME_BARRIER_COST,
        neginf=1.0
    )
    if meta['nodata'] is not None:
        resistance_array[resistance_array == meta['nodata']] = EXTREME_BARRIER_COST
    resistance_array[resistance_array <= 0] = 1.0
    
    # For plotting, set barriers to 'nan' so they are white
    resistance_plot = resistance_array.copy().astype(float)
    resistance_plot[resistance_plot == EXTREME_BARRIER_COST] = np.nan

Loading base resistance raster: c:\ZHAW\5.Semester\PA2\PA2-Modelling_Wildlife_Corridors\results\final_resistance_surface.tif


In [89]:
# --- 3. Load Traffic Raster (for corridor overlay) ---
print(f"Loading traffic raster: {TRAFFIC_RASTER}")
with rasterio.open(TRAFFIC_RASTER) as src:
    traffic_array = src.read(1).astype(np.float32)
    
    # Find the threshold value (e.g., P90)
    non_zero_pixels = traffic_array[traffic_array > 0]
    if non_zero_pixels.size == 0:
        print("Error: No traffic data found.")
        sys.exit(1)
        
    traffic_threshold_val = np.percentile(non_zero_pixels, TRAFFIC_PERCENTILE_THRESHOLD)
    print(f"Traffic threshold (P{TRAFFIC_PERCENTILE_THRESHOLD}) is: {traffic_threshold_val} crossings")
    
    # Create a masked array to hide all non-corridor pixels
    traffic_masked = np.ma.masked_less(traffic_array, traffic_threshold_val)

Loading traffic raster: c:\ZHAW\5.Semester\PA2\PA2-Modelling_Wildlife_Corridors\results\final_corridor_traffic.tif
Traffic threshold (P99.0) is: 35934.0 crossings


In [90]:
# --- 4. Create the Bottleneck Map and Coordinates ---

# Create a mask where traffic is BELOW our threshold (these areas are not part of the core corridor)
mask = traffic_array < traffic_threshold_val

# Create our output array by copying the base resistance values
corridor_bottlenecks = resistance_array.copy()

# Apply the mask: outside the core corridor, set the value to NoData
corridor_bottlenecks[mask] = NODATA_VALUE
print("Bottleneck mask applied.")

# *************************************************************************
# *** CORRECTED CODE ADDED: IDENTIFY COORDINATES FOR PLOTTING ***
# *************************************************************************

# 1. Identify bottleneck pixels: where cost is high (>= threshold) AND is inside the corridor
bottleneck_condition = (corridor_bottlenecks >= BOTTLENECK_COST_THRESHOLD)

# 2. np.where finds the (row, col) indices of all pixels meeting the condition
bottleneck_indices = np.where(bottleneck_condition)

# 3. Extract (row, col) coordinates for the scatter plot (these are the missing variables!)
rows = bottleneck_indices[0]
cols = bottleneck_indices[1]

print(f"Identified {len(rows)} bottleneck pixels with cost >= {BOTTLENECK_COST_THRESHOLD} for scatter plot.")

# *************************************************************************

# --- 5. Save the New Raster ---
# Update the metadata for our new output file
meta.update({
    'dtype': 'float32',
    'nodata': NODATA_VALUE
})

print(f"Saving bottleneck map to {BOTTLENECK_RASTER}...")
with rasterio.open(BOTTLENECK_RASTER, 'w', **meta) as dst:
    dst.write(corridor_bottlenecks.astype(np.float32), 1)

print("Analysis complete.")

Bottleneck mask applied.
Identified 15 bottleneck pixels with cost >= 5000.0 for scatter plot.
Saving bottleneck map to c:\ZHAW\5.Semester\PA2\PA2-Modelling_Wildlife_Corridors\results\corridor_bottlenecks.tif...


CPLE_AppDefinedError: Deleting c:\ZHAW\5.Semester\PA2\PA2-Modelling_Wildlife_Corridors\results\corridor_bottlenecks.tif failed: Permission denied

In [95]:
# --- 5. Create the Composite Plot ---
print("Generating composite plot...")
fig, ax = plt.subplots(figsize=(15, 15))

# --- Layer 1: Base Resistance ---
cmap_base = plt.colormaps.get('Greys').copy()
cmap_base.set_bad(color='white') # Show 'nan' (barriers) as white
vmax_resistance = np.nanpercentile(resistance_plot, 99) # Get a good max value
im_base = ax.imshow(
    resistance_plot, 
    cmap=cmap_base, 
    norm=colors.LogNorm(vmin=1, vmax=vmax_resistance)
)
# Add a colorbar for the base map
cbar_base = fig.colorbar(im_base, ax=ax, shrink=0.5, pad=0.02, label='Resistance Cost (Log Scale)')

# --- Layer 2: Corridor Overlay ---
cmap_traffic = plt.colormaps.get('viridis').copy()
cmap_traffic.set_bad(color='none') # Make non-corridor pixels transparent
im_traffic = ax.imshow(
    traffic_masked, 
    cmap=cmap_traffic,
    norm=colors.LogNorm(vmin=traffic_threshold_val, vmax=traffic_array.max()),
    alpha=0.7 # Make it semi-transparent
)
# Add a colorbar for the traffic
cbar_traffic = fig.colorbar(im_traffic, ax=ax, shrink=0.5, pad=0.1, label='LCP Traffic (Top 10%)')



# --- Layer 3: Bottleneck Circles ---
ax.scatter(
    cols, 
    rows, 
    s=80,             # Size of the circles
    facecolors='none',# Hollow circles
    edgecolors='red', # Red outline
    linewidths=0.8,   # Thin outline
    label=f'Bottleneck (Cost > {BOTTLENECK_COST_THRESHOLD})'
)

Generating composite plot...


In [96]:
# --- 6. Finalize and Save ---
ax.set_title("Final Composite Map: Corridors and Bottlenecks", fontsize=20)
ax.set_xlabel('Easting (Pixel Coordinates)')
ax.set_ylabel('Northing (Pixel Coordinates)')
ax.legend(loc='upper right', facecolor='white', framealpha=0.9, markerscale=1.5)
plt.tight_layout()

print(f"Saving final plot to {OUTPUT_PLOT}...")
plt.savefig(OUTPUT_PLOT, dpi=150, bbox_inches="tight")

print("Done.")

Saving final plot to c:\ZHAW\5.Semester\PA2\PA2-Modelling_Wildlife_Corridors\results\final_annotated_composite_map.png...
Done.


In [104]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

CROPPED_PLOT = RESULTS_DIR / "final_annotated_composite_map_cropped.png"

# --- DEFINE ZOOM EXTENT ---
# Rows (Northing) and Columns (Easting) indices for the zoom window
ROW_MIN = 2200
ROW_MAX = 2400
COL_MIN = 2100
COL_MAX = 2300

# --- 1. Filter and Slice Data for Zoom ---

# Slice the Resistance and Traffic Rasters (500x500 arrays)
resistance_plot_zoom = resistance_plot[ROW_MIN:ROW_MAX, COL_MIN:COL_MAX]
traffic_masked_zoom = traffic_masked[ROW_MIN:ROW_MAX, COL_MIN:COL_MAX]

# Filter the Bottleneck Coordinates
# 1. Create a boolean mask for points strictly inside the specified box
scatter_mask = (rows >= ROW_MIN) & (rows < ROW_MAX) & (cols >= COL_MIN) & (cols < COL_MAX)

# 2. Apply the mask and adjust coordinates to be RELATIVE to the zoomed area (start index 0)
rows_zoom = rows[scatter_mask] - ROW_MIN
cols_zoom = cols[scatter_mask] - COL_MIN

print(f"Zooming to index: Rows {ROW_MIN}-{ROW_MAX}, Cols {COL_MIN}-{COL_MAX}")
print(f"Plot size is {resistance_plot_zoom.shape[0]}x{resistance_plot_zoom.shape[1]} pixels.")
print(f"Plotting {len(rows_zoom)} bottleneck circles in the zoomed area.")


# --- 5. Create the Composite Plot (Adjusted) ---
print("Generating composite plot...")
fig, ax = plt.subplots(figsize=(15, 15))

# --- Layer 1: Base Resistance ---
cmap_base = plt.colormaps.get('Greys').copy()
cmap_base.set_bad(color='white') 
# Recalculate vmax based on the zoomed resistance slice for proper contrast
vmax_resistance = np.nanpercentile(resistance_plot_zoom, 99) 
# Use the sliced array
im_base = ax.imshow(
    resistance_plot_zoom, 
    cmap=cmap_base, 
    norm=colors.LogNorm(vmin=1, vmax=vmax_resistance)
)
cbar_base = fig.colorbar(im_base, ax=ax, shrink=0.5, pad=0.02, label='Resistance Cost (Log Scale)')

# --- Layer 2: Corridor Overlay ---
cmap_traffic = plt.colormaps.get('viridis').copy()
cmap_traffic.set_bad(color='none')
# Use the sliced array
im_traffic = ax.imshow(
    traffic_masked_zoom, 
    cmap=cmap_traffic,
    # Use traffic_array.max() to keep color intensity consistent with the overall model
    norm=colors.LogNorm(vmin=traffic_threshold_val, vmax=traffic_array.max() if traffic_array.size > 0 else traffic_threshold_val + 1),
    alpha=0.7
)
cbar_traffic = fig.colorbar(im_traffic, ax=ax, shrink=0.5, pad=0.1, label='LCP Traffic (Top 10%)')

# --- Layer 3: Bottleneck Circles ---
# Use the filtered and adjusted coordinates (rows_zoom, cols_zoom)
ax.scatter(
    cols_zoom, 
    rows_zoom, 
    s=500,
    facecolors='none',
    edgecolors='red',
    linewidths=3,
    label=f'Bottleneck (Cost > {BOTTLENECK_COST_THRESHOLD})'
)
    
# --- 6. Finalize and Save ---
# Adjust labels to reflect the true indices being plotted
ax.set_title(f"Composite Map: Corridors and Bottlenecks (Cropped)", fontsize=16)
ax.set_xlabel(f'Easting (Pixel Indices {COL_MIN} to {COL_MAX})')
ax.set_ylabel(f'Northing (Pixel Indices {ROW_MIN} to {ROW_MAX})')

# Ensure the plot area is correctly centered on the 500x500 sliced area
ax.set_xlim(-0.5, resistance_plot_zoom.shape[1] - 0.5)
ax.set_ylim(resistance_plot_zoom.shape[0] - 0.5, -0.5) # Reverse y-axis for GIS convention

ax.legend(loc='upper right', facecolor='white', framealpha=0.9, markerscale=0.5)

plt.tight_layout()

print(f"Saving final plot to {CROPPED_PLOT}...")
plt.savefig(CROPPED_PLOT, dpi=150, bbox_inches="tight")

print("Done.")

Zooming to index: Rows 2200-2400, Cols 2100-2300
Plot size is 200x200 pixels.
Plotting 1 bottleneck circles in the zoomed area.
Generating composite plot...
Saving final plot to c:\ZHAW\5.Semester\PA2\PA2-Modelling_Wildlife_Corridors\results\final_annotated_composite_map_cropped.png...
Done.
